# Introdução ao Scikit Learn

Scikit-learn (Sklearn) é uma biblioteca voltada para o campo de `machine learning` em `Python`. Assim, a mesma possui diversas funcionalidas já prontas para uso em machine learning e estatística, incluindo classificação, regressão, clustering e mais.

Este notebook tem como foco apenas alguns estudos envolvendo alguns conceitos básicos de aprendizado de máquina e ao sklearn.

## Importações

   - [pandas](https://pandas.pydata.org/): biblioteca voltada para manipulação e análise de dados em Python.
   - [datetime](https://docs.python.org/3/library/datetime.html): modulo para tratar estrutura de dados envolvendo tempo.
   - [numpy](https://numpy.org/): biblioteca para matemática avançada em Python.
   - [sklearn](https://scikit-learn.org/stable/): biblioteca para aprendizado de máquina em Python.
   - [warnings](https://docs.python.org/3/library/warnings.html): para ignorar alguns warnings sendo exibidos.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from datetime import datetime

## Base da dados

Usaremos uma base de dados disponibilizada nesse [link](https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv).

Este arquivo será importando para uma estrutura `DataFrame` utilizando a biblioteca `pandas` a fim de possibilitar sua manipulação e análise.

In [2]:
# obtendo o data frame
uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"
df = pd.read_csv(uri)

## Análise dos dados

A primeira coisa que faremos após importar o dataset será examinar as dimensões do DataFrame e as primeiras entradas. Isso possibilitará criar uma consciência situacional inicial a respeito do formato de entrada e da estrutura geral dos dados.

In [3]:
# dimensões do df
print("DIMENSÕES DO DATAFRAME:")
print("Linhas:\t\t{}".format(df.shape[0]))
print("Colunas:\t{}".format(df.shape[1]))

DIMENSÕES DO DATAFRAME:
Linhas:		10000
Colunas:	5


Já as primeiras 5 entradas desse conjunto de dados permitem adquirirmos uma noção inicial sobre o tipo de formato, exemplos de entrada e formulação das hipóteses iniciais do processo investigativo.

In [4]:
# ver as 5 primeiras entradas
df.head()

,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


Vamos renomear os nomes das colunas do dataset para o português, buscando uma melhor entendibilidade.

In [5]:
# dicionário tendo coluna:nova_coluna
novos_nomes = {
    "mileage_per_year": "milhas_por_ano",
    "model_year": "ano_do_modelo",
    "price": "preco",
    "Unnamed: 0": "id",
    "sold": "vendido"
}

# renomeando
df = df.rename(columns=novos_nomes)

# exibindo
df.head(1)

,id,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,yes


Observando que a coluna `vendido` identificado como uma `classe/label`  possui como valores 'yes/no', vamos tratar para que seja 0 ou 1.

In [6]:
# dicionário tendo valor:nova_valor
troca_vendido = {
    'no': 0,
    'yes': 1
}

# aplicando o map
df.vendido = df.vendido.map(troca_vendido)

# exibindo
df.head(3)

,id,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0


Observando as colunas:
    
    milhas_por_ano
    ano_do_modelo
    preco

Pode-se inferir que a partir delas pode-se obter novos dados, como, por exemplo, a idade do modelo em relação ao ano atual e também transformar milhas para kilometros.

In [7]:
# busco o ano atual
ano_atual = datetime.today().year

# crio uma nova feature mais relevante
df['idade_do_modelo'] = ano_atual - df.ano_do_modelo

# outra feature mais relevante
df['km_por_ano'] = df.milhas_por_ano*1.60934

# removo as colunas destacadas
df = df.drop(columns=['id', 'milhas_por_ano', 'ano_do_modelo'], axis=1)

# exibo
df.head(6)

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,21,35085.22134
1,40557.96,1,23,12622.05362
2,89627.50,0,15,11440.79806
3,95276.14,0,6,43167.32682
4,117384.68,1,7,12770.11290
5,55405.26,1,14,23594.53374


Agora, iremos exibir algumas informações explorátorias sobre o dataset, dessa maneira, observando que o mesmo está em um bom estado para começarmos a montar o modelo de aprendizado de máquina.

In [8]:
# tipo das variáveis
print("\nTIPOS DAS VARIÁVEIS:")
print(df.dtypes)


TIPOS DAS VARIÁVEIS:
preco              float64
vendido              int64
idade_do_modelo      int64
km_por_ano         float64
dtype: object


In [9]:
# valores ausentes
print("\nVALORES AUSENTES:")
print((df.isnull().sum() / df.shape[0]).sort_values(ascending=False))


VALORES AUSENTES:
km_por_ano         0.0
idade_do_modelo    0.0
vendido            0.0
preco              0.0
dtype: float64


In [10]:
# valores únicos
print("\nVALORES ÚNICOS:")
print(df.nunique().sort_values())


VALORES ÚNICOS:
vendido               2
idade_do_modelo      20
km_por_ano         7605
preco              9997
dtype: int64


## Avaliando modelos

Aqui iremos avaliar alguns modelos que melhor se encaixam na problemática.

Primeiramente, iremos destacar algumas constantes que serão utilizadas a partir da célular abaixo.

In [11]:
# dados referentes as fetures
x = df[["preco", "idade_do_modelo","km_por_ano"]]

# label com o resultado esperado - classificado
y = df["vendido"]

# é a semente
SEED = 5

# definindo a semente para o atributo random_state dos métodos do sklearn
np.random.seed(SEED)

# dividindo o dataframe em treino/teste
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)

# exibindo
print(f'Treinaremos com {len(treino_x)} elementos e testaremos com {len(teste_x)} elementos.')

Treinaremos com 7500 elementos e testaremos com 2500 elementos.


O primeiro modelo a ser testado será o LinearSVC. Basicamente, o que esse algoritmo faz é encontrar uma linha que separe as classes.

In [12]:
# criando o modelo
modelo = LinearSVC(random_state=SEED)

# treinando o modelo
modelo.fit(treino_x, treino_y)

# realizando a previsão
previsoes = modelo.predict(teste_x)

# medindo a acurácia
acuracia = accuracy_score(teste_y, previsoes) * 100

# exibindo
print(f'A acurácia foi de {acuracia}%.')

A acurácia foi de 48.16%.


É nítido que o modelo foi mal condicionado, pelo baixo resultado. Entretanto,pode-se verificar um teste utilizando uma técnica de algoritmo base, por meio do `DummyClassifier`. O algoritmo base converte uma coluna deixando somente 0 ou somente 1. Com isso nós implementamos o teste do algoritmo e obtemos o resultado. Como o próprio método nos diz, é burro, ou seja, deve ser utilizado apenas como parâmetro.

In [13]:
# criando o dummy preservando a seed e a strategy=stratified
dummy_stratified = DummyClassifier(random_state=SEED, strategy='stratified')

# treinando o dummy
dummy_stratified.fit(treino_x, treino_y)

# medindo a acurácia
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

# exibindo
print(f'A acurácia foi de {acuracia}%.')

A acurácia foi de 51.239999999999995%.


Percebe-se que utilizando o tese pelo `dummy` o resultado foi melhor, o que implica que o método LinearSVC não é correto para tratar a problemática. Assim, iremos utilizar o método abaixo com o `StandardScaler`.

- `StandardScaler`: Transforma os dados de maneira que tenha média como 0 e desvio padrão como 1. Em resumo, padroniza os dados . A padronização é útil para dados que possuem valores negativos. Ele organiza os dados em uma distribuição normal padrão . É mais útil na classificação do que na regressão .

In [14]:
# criando dados de treino/testes que serão modifcados, por isso 'raw'
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)

# criando o standardScaler
scaler = StandardScaler()

# treinando
scaler.fit(raw_treino_x)

# realizando a tranformação de cru para os dados que realmente serão utilizados pelo modelo
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

# importando o modelo
modelo = SVC()
# treinando
modelo.fit(treino_x, treino_y)
# predizendo
previsoes = modelo.predict(teste_x)
# acuracia
acuracia = accuracy_score(teste_y, previsoes) * 100

# exibindo
print(f'A acurácia foi de {acuracia}%.')

A acurácia foi de 76.6%.


Também, podemos avaliar utilizando o método de `DecisionTree`.

In [15]:
# criando dados de treino/testes que serão modifcados, por isso 'raw'
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,stratify = y)

# importando o modelo
modelo = DecisionTreeClassifier(max_depth=3)
# treinamento
modelo.fit(raw_treino_x, treino_y)
# previsao
previsoes = modelo.predict(raw_teste_x)
# acuracia
acuracia = accuracy_score(teste_y, previsoes) * 100

# exibindo
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 79.36%
